In [111]:
import json

In [112]:
path = "/home/northja/datasets/mcsa/mcsa.json"

In [113]:
vals = None

with open(path, "r") as read_file:
    vals = json.load(read_file)

#    for key,value in vals:
    

In [114]:
vals.keys()

dict_keys(['count', 'next', 'previous', 'results'])

## Download from mCSA

In [236]:
mcsa_id = 50

In [209]:
steps = vals['results'][mcsa_id]['reaction']['mechanisms'][0]['steps']

In [210]:
steps[0]['marvin_xml']

'www.ebi.ac.uk/thornton-srv/m-csa/media/schemes/Step_macie.31.1.1_lNjVSkW.mrv'

In [211]:
len(steps)

7

In [212]:
urllist = [steps[i]['marvin_xml'] for i in list(range(len(steps)))]

In [213]:
urllist

['www.ebi.ac.uk/thornton-srv/m-csa/media/schemes/Step_macie.31.1.1_lNjVSkW.mrv',
 'www.ebi.ac.uk/thornton-srv/m-csa/media/schemes/Step_macie.31.1.2_Zv3Ezta.mrv',
 'www.ebi.ac.uk/thornton-srv/m-csa/media/schemes/Step_macie.31.1.3_eMVYdZu.mrv',
 'www.ebi.ac.uk/thornton-srv/m-csa/media/schemes/Step_macie.31.1.4_kPho2vr.mrv',
 'www.ebi.ac.uk/thornton-srv/m-csa/media/schemes/Step_macie.31.1.5_Gd6kTIj.mrv',
 'www.ebi.ac.uk/thornton-srv/m-csa/media/schemes/macie_entry_31_1_6.mrv',
 'www.ebi.ac.uk/thornton-srv/m-csa/media/schemes/Step_macie.31.1.7.mrv']

In [214]:
import urllib3
http = urllib3.PoolManager()
from pathlib import Path

In [215]:
datasetpath = f"/home/northja/datasets/mcsa/entries/entry_{mcsa_id}/"

In [216]:
Path(datasetpath).mkdir(parents=True, exist_ok=True)

In [217]:
print("Downloading with urllib...")
N = 0

local_paths = []

for url in urllist:
    r = http.request('GET',url)
    Name =str(N+1) 
    localpath = f"{datasetpath}mcsa_{mcsa_id}_step_{N+1}.mrv"
    with open(localpath, "wb") as fcont:
        fcont.write(r.data)
    N += 1

    local_paths.append(localpath)

In [244]:
import urllib3
http = urllib3.PoolManager()
from pathlib import Path
from openbabel import openbabel

all = vals['results']
count = len(all)

for mcsa_id in list(range(count)):
    steps = vals['results'][mcsa_id]['reaction']['mechanisms'][0]['steps']
    urllist = [steps[i]['marvin_xml'] for i in list(range(len(steps)))]
    
    datasetpath = f"/home/northja/datasets/mcsa/entries/entry_{mcsa_id}/"

    Path(datasetpath).mkdir(parents=True, exist_ok=True)

    ## DOWNLOAD FILES
    print("Downloading with urllib...")
    N = 0

    local_paths = []

    for url in urllist: 
        localpath = f"{datasetpath}mcsa_{mcsa_id}_step_{N+1}.mrv"
        local_paths.append(localpath)

        if Path(localpath).exists():
            break

        r = http.request('GET',url)
        Name =str(N+1) 
        with open(localpath, "wb") as fcont:
            fcont.write(r.data)
        N += 1

        
        print(f"DONE: {url}")
    
    obConversion = openbabel.OBConversion()
    obConversion.SetInAndOutFormats("mrv", "sdf")

    local_sdfs = []

    for file in local_paths:
        mol = openbabel.OBMol()
        obConversion.ReadFile(mol, file)   # Open Babel will uncompress automatically

        mol.AddHydrogens()

        print(f"ATOMS:\t {mol.NumAtoms()}\tbonds: {mol.NumBonds()}\tres: {mol.NumResidues()}")

        molname = file
        molname = molname.replace(".mrv", ".sdf")

        obConversion.WriteFile(mol, molname)

        local_sdfs.append(molname)

    graphs = []

    for sdf in local_sdfs:
        with SDFRead(open(sdf, 'r')) as f:
            sdf = next(f)
            graphs.append(sdf)

ATOMS:	 84	bonds: 78	res: 0
ATOMS:	 135	bonds: 128	res: 0


StopIteration: 

## Setup dataset

In [218]:
datasetpath

'/home/northja/datasets/mcsa/entries/entry_30/'

In [220]:
obConversion = openbabel.OBConversion()
obConversion.SetInAndOutFormats("mrv", "sdf")

local_sdfs = []

for file in local_paths:
    mol = openbabel.OBMol()
    obConversion.ReadFile(mol, file)   # Open Babel will uncompress automatically

    mol.AddHydrogens()

    print(f"ATOMS:\t {mol.NumAtoms()}\tbonds: {mol.NumBonds()}\tres: {mol.NumResidues()}")

    molname = file
    molname = molname.replace(".mrv", ".sdf")

    obConversion.WriteFile(mol, molname)

    local_sdfs.append(molname)

ATOMS:	 180	bonds: 181	res: 0
ATOMS:	 183	bonds: 182	res: 0
ATOMS:	 183	bonds: 183	res: 0
ATOMS:	 186	bonds: 186	res: 0
ATOMS:	 186	bonds: 186	res: 0
ATOMS:	 186	bonds: 185	res: 0
ATOMS:	 186	bonds: 184	res: 0


In [221]:
from CGRtools.files import SDFRead # import all available readers and writers

In [222]:
local_sdfs

['/home/northja/datasets/mcsa/entries/entry_30/mcsa_30_step_1.sdf',
 '/home/northja/datasets/mcsa/entries/entry_30/mcsa_30_step_2.sdf',
 '/home/northja/datasets/mcsa/entries/entry_30/mcsa_30_step_3.sdf',
 '/home/northja/datasets/mcsa/entries/entry_30/mcsa_30_step_4.sdf',
 '/home/northja/datasets/mcsa/entries/entry_30/mcsa_30_step_5.sdf',
 '/home/northja/datasets/mcsa/entries/entry_30/mcsa_30_step_6.sdf',
 '/home/northja/datasets/mcsa/entries/entry_30/mcsa_30_step_7.sdf']

In [223]:
graphs = []

for sdf in local_sdfs:
    with SDFRead(open(sdf, 'r')) as f:
        sdf = next(f)
        graphs.append(sdf)

In [224]:
from CGRtools.containers import * # import all containers
from CGRtools.containers import MoleculeContainer
from CGRtools.containers.bonds import Bond

In [233]:
r = ReactionContainer(reactants=[graphs[4]], products=[graphs[3]]) # one-step way to construct reaction

In [234]:
cgr = ~r # CGR from reactions
# or 
cgr = r.compose()
print(cgr) # signature is printed out.
#cgr.clean2d()
cgr

[H]C([H])([H])C([H])([H])C([H])([H])C([H])([H])N([H])C(=[N+]([H])[H])N([H])[H].[H]C([H])([H])C([H])([H])C1=C(N([H])C=2C([H])=C([H])C(=C([H])C1=2)[H])[H].[H]C([H])([H])C([H])([H])C([H])([H])C(=O)[O->0][.>-][H][->.]O([H])[.>-][H][->.]O[->=]C3[=>-]C(C(SC([H])([H])C([H])([H])[H])(N(C(=O)N3[H])C4(OC(C([H])(O[H])C4([H])[H])([H])C([H])([H])OP([O-])([O-])=O)[H])[H])([.>-][H][->.]O([.>-][H][->.][O0>-]C=5C([H])=C(C(=C([H])C=5[H])C([H])([H])C([H])([H])[H])[H])[H])C([H])([H])N6C(C([H])([H])N(C=7N=C(N([H])C(C6=7)=O)N([H])[H])[H])([H])C([H])([H])N(C=8C(=C([H])C(C(N([H])C(C([H])([H])C([H])([H])C(=O)[O-])([H])C([O-])=O)=O)=C([H])C=8[H])[H])[H].[H]C([H])([H])C([H])([H])C([O-])=O
